# Lab 1: Configuração do Banco de Dados (PostgreSQL)

Este notebook executa a configuração do banco de dados PostgreSQL de forma interativa. As células de código a seguir irão se conectar ao contêiner do banco de dados e executar os scripts de criação de tabelas (DDL) e inserção de dados (DML) usando Python.

## Inicializar os bancos de dados

Etapa obrigatória, pois todos os serviços iniciam desligados para que o MYBinder consiga inicializar o Container.

In [ ]:
!/usr/local/bin/entrypoint.sh

## 1. Conexão e Execução dos Scripts

A célula abaixo contém todo o processo:
1.  Importa as bibliotecas necessárias.
2.  Define as queries DDL e DML.
3.  Estabelece uma conexão com o contêiner `petshop_db` (o nome do serviço no `docker-compose`).
4.  Cria um cursor e executa as queries.
5.  Confirma as transações e fecha a conexão.

In [ ]:
import psycopg2
import os

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

DDL_SCRIPT = """
-- =========== CRIAÇÃO DA FUNÇÃO DE TRIGGER ===========
-- (Deve ser executada antes dos CREATE TABLEs que a utilizam)

CREATE OR REPLACE FUNCTION update_last_modified_column()
RETURNS TRIGGER AS $$
BEGIN
   NEW.dlastupdate = NOW(); 
   RETURN NEW;
END;
$$ LANGUAGE plpgsql;


-- =========== CRIAÇÃO DE TIPOS ENUMERADOS (ENUMs) ===========
-- Melhora a integridade dos dados para colunas com valores restritos.

DO $$
BEGIN
    IF NOT EXISTS (SELECT 1 FROM pg_type WHERE typname = 'enum_booking_status') THEN
        CREATE TYPE enum_booking_status AS ENUM ('Agendado', 'Realizado', 'Cancelado', 'Pendente');
    END IF;
    IF NOT EXISTS (SELECT 1 FROM pg_type WHERE typname = 'enum_pet_species') THEN
        CREATE TYPE enum_pet_species AS ENUM ('Cão', 'Gato');
    END IF;
    IF NOT EXISTS (SELECT 1 FROM pg_type WHERE typname = 'enum_vaccine_target_species') THEN
        CREATE TYPE enum_vaccine_target_species AS ENUM ('Cão', 'Gato', 'Ambos');
    END IF;
    IF NOT EXISTS (SELECT 1 FROM pg_type WHERE typname = 'enum_execution_history_status') THEN
        CREATE TYPE enum_execution_history_status AS ENUM ('RUNNING', 'COMPLETED', 'FAILED');
    END IF;
END$$;


-- =========== TABELAS PRINCIPAIS ===========

CREATE TABLE IF NOT EXISTS organization_invite (
    organization_invite_id SERIAL PRIMARY KEY,
    invite_code VARCHAR(500) UNIQUE,
    expiration_date TIMESTAMP WITH TIME ZONE,
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_organization_invite_dlastupdate ON organization_invite;
CREATE TRIGGER update_organization_invite_dlastupdate BEFORE UPDATE ON organization_invite FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS organization (
    organization_id SERIAL PRIMARY KEY,
    name VARCHAR(80) NOT NULL UNIQUE,
    social_name VARCHAR(80) NOT NULL,
    description VARCHAR(255) NOT NULL,
    identification_code VARCHAR(20) UNIQUE,
    links TEXT[],
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_organization_dlastupdate ON organization;
CREATE TRIGGER update_organization_dlastupdate BEFORE UPDATE ON organization FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();


CREATE TABLE IF NOT EXISTS organization_apikey (
    organization_id INTEGER NOT NULL REFERENCES organization(organization_id),
    api_key VARCHAR(500) NOT NULL,
    PRIMARY KEY (organization_id, api_key),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_organization_apikey_dlastupdate ON organization_apikey;
CREATE TRIGGER update_organization_apikey_dlastupdate BEFORE UPDATE ON organization_apikey FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS tutor (
    tutor_id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL UNIQUE,
    phone VARCHAR(20) UNIQUE,

    organization_id INTEGER REFERENCES organization(organization_id),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_tutor_dlastupdate ON tutor;
CREATE TRIGGER update_tutor_dlastupdate BEFORE UPDATE ON tutor FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS pet (
    pet_id SERIAL PRIMARY KEY,
    tutor_id INTEGER NOT NULL REFERENCES tutor(tutor_id),
    name VARCHAR(100) NOT NULL,
    image_path TEXT,
    birth_date DATE,
    ignore_recommendation BOOLEAN DEFAULT FALSE,

    -- classification columns
    species enum_pet_species NOT NULL,
    animal_type VARCHAR(50),
    fur_type VARCHAR(50),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_pet_dlastupdate ON pet;
CREATE TRIGGER update_pet_dlastupdate BEFORE UPDATE ON pet FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();


-- =========== TABELAS DE PRODUTOS E COMPRAS ===========

CREATE TABLE IF NOT EXISTS product (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    category VARCHAR(100),

    organization_id INTEGER REFERENCES organization(organization_id),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_product_dlastupdate ON product;
CREATE TRIGGER update_product_dlastupdate BEFORE UPDATE ON product FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS purchase (
    purchase_id SERIAL PRIMARY KEY,
    tutor_id INTEGER NOT NULL REFERENCES tutor(tutor_id),
    product_id INTEGER NOT NULL REFERENCES product(product_id),

    purchase_date TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    quantity INTEGER NOT NULL,
    price NUMERIC(10, 2) NOT NULL,
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_purchase_dlastupdate ON purchase;
CREATE TRIGGER update_purchase_dlastupdate BEFORE UPDATE ON purchase FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();


-- =========== TABELAS DE VACINAÇÃO ===========

CREATE TABLE IF NOT EXISTS vaccine_reference (
    vaccine_reference_id SERIAL PRIMARY KEY,
    vaccine_name VARCHAR(150) NOT NULL UNIQUE,
    description TEXT NOT NULL,
    target_species enum_vaccine_target_species NOT NULL,

    mandatory BOOLEAN NOT NULL DEFAULT FALSE,
    first_dose_age_months NUMERIC(5, 2) NOT NULL,
    booster_interval_months NUMERIC(5, 2),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_vaccine_reference_dlastupdate ON vaccine_reference;
CREATE TRIGGER update_vaccine_reference_dlastupdate BEFORE UPDATE ON vaccine_reference FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

-- Tabela de associação N:N para vacinas equivalentes
CREATE TABLE IF NOT EXISTS vaccine_equivalence (
    vaccine_id INTEGER NOT NULL REFERENCES vaccine_reference(vaccine_reference_id) ON DELETE CASCADE,
    equivalent_vaccine_id INTEGER NOT NULL REFERENCES vaccine_reference(vaccine_reference_id) ON DELETE CASCADE,
    PRIMARY KEY (vaccine_id, equivalent_vaccine_id),

    -- Garante que a relação não seja duplicada (ex: A->B e B->A)
    CHECK (vaccine_id < equivalent_vaccine_id),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_vaccine_equivalence_dlastupdate ON vaccine_equivalence;
CREATE TRIGGER update_vaccine_equivalence_dlastupdate BEFORE UPDATE ON vaccine_equivalence FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS vaccination_record (
    vaccination_record_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),
    vaccine_reference_id INTEGER NOT NULL REFERENCES vaccine_reference(vaccine_reference_id),

    application_date DATE NOT NULL,
    vaccine_batch VARCHAR(100),
    responsible_vet VARCHAR(255),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_vaccination_record_dlastupdate ON vaccination_record;
CREATE TRIGGER update_vaccination_record_dlastupdate BEFORE UPDATE ON vaccination_record FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS vaccine_recommendation (
    vaccine_recommendation_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),

    --vaccine_reference_id INTEGER NOT NULL REFERENCES vaccine_reference(vaccine_reference_id),
    vaccine_name VARCHAR(255),
    description TEXT,
    mandatory BOOLEAN,

    suggested_date DATE NOT NULL,
    ignore_recommendation BOOLEAN DEFAULT FALSE,

    -- Garante uma recomendação única por pet/vacina/data
    --UNIQUE (pet_id, vaccine_reference_id, suggested_date)
    UNIQUE (pet_id, suggested_date),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_vaccine_recommendation_dlastupdate ON vaccine_recommendation;
CREATE TRIGGER update_vaccine_recommendation_dlastupdate BEFORE UPDATE ON vaccine_recommendation FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();


-- =========== TABELAS DE AGENDAMENTO (BOOKING) ===========

CREATE TABLE IF NOT EXISTS booking (
    booking_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),

    -- NOTA: Idealmente, 'service_type' seria uma FK para uma tabela 'service_catalog'
    service_type VARCHAR(100) NOT NULL, 

    booking_date TIMESTAMP WITH TIME ZONE NOT NULL,
    status enum_booking_status NOT NULL,
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_booking_dlastupdate ON booking;
CREATE TRIGGER update_booking_dlastupdate BEFORE UPDATE ON booking FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS booking_reference (
    booking_reference_id SERIAL PRIMARY KEY,
    frequency_days INTEGER NOT NULL,

    -- classification columns
    species enum_pet_species NOT NULL,
    animal_type VARCHAR(50),
    fur_type VARCHAR(50),

    -- Garante a regra de negócio da PK original (que tinha colunas nulas)
    UNIQUE (species, animal_type, fur_type),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_booking_reference_dlastupdate ON booking_reference;
CREATE TRIGGER update_booking_reference_dlastupdate BEFORE UPDATE ON booking_reference FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS booking_recommendation (
    booking_recommendation_id SERIAL PRIMARY KEY,
    pet_id INTEGER NOT NULL REFERENCES pet(pet_id),

    --service_type VARCHAR(100) NOT NULL, 

    suggested_date DATE NOT NULL,
    average_frequency_days INTEGER,
    ignore_recommendation BOOLEAN DEFAULT FALSE,

    -- Corrigida PK: permite uma recomendação ativa por pet/serviço
    --UNIQUE (pet_id, service_type)
    UNIQUE (pet_id),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_booking_recommendation_dlastupdate ON booking_recommendation;
CREATE TRIGGER update_booking_recommendation_dlastupdate BEFORE UPDATE ON booking_recommendation FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();


-- =========== TABELAS DE RELATÓRIO E EXECUÇÃO ===========

CREATE TABLE IF NOT EXISTS ltv_by_pet_profile (
    ltv_by_pet_profile_id SERIAL PRIMARY KEY,

    -- classification columns
    species enum_pet_species NOT NULL,
    animal_type VARCHAR(50),
    fur_type VARCHAR(50),

    total_value NUMERIC(10, 2) NOT NULL DEFAULT 0,

    -- Chave de negócio para identificar o perfil
    UNIQUE (species, animal_type, fur_type),
    
    -- Colunas de Auditoria
    dcreated TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    dlastupdate TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    nenabled BOOLEAN NOT NULL DEFAULT TRUE
);
DROP TRIGGER IF EXISTS update_ltv_by_pet_profile_dlastupdate ON ltv_by_pet_profile;
CREATE TRIGGER update_ltv_by_pet_profile_dlastupdate BEFORE UPDATE ON ltv_by_pet_profile FOR EACH ROW EXECUTE PROCEDURE update_last_modified_column();

CREATE TABLE IF NOT EXISTS execution_history (
    execution_id SERIAL PRIMARY KEY,
    target_table VARCHAR(255) NOT NULL,
    start_time TIMESTAMP WITH TIME ZONE NOT NULL DEFAULT NOW(),
    end_time TIMESTAMP WITH TIME ZONE,
    status enum_execution_history_status NOT NULL,
    error_message TEXT,
    records_processed INTEGER
    
    -- Colunas de Auditoria (dcreated/dlastupdate/nenabled) omitidas, pois start_time/end_time já servem para rastrear.
);


-- =========== CRIAÇÃO DE ÍNDICES (CRUCIAL PARA PERFORMANCE) ===========
-- Índices em Chaves Estrangeiras (FKs) e colunas de filtro (WHERE).

CREATE INDEX IF NOT EXISTS idx_tutor_organization_id ON tutor(organization_id);

CREATE INDEX IF NOT EXISTS idx_pet_tutor_id ON pet(tutor_id);

CREATE INDEX IF NOT EXISTS idx_purchase_tutor_id ON purchase(tutor_id);
CREATE INDEX IF NOT EXISTS idx_purchase_product_id ON purchase(product_id);

CREATE INDEX IF NOT EXISTS idx_vaccination_record_pet_id ON vaccination_record(pet_id);
CREATE INDEX IF NOT EXISTS idx_vaccination_record_vaccine_ref_id ON vaccination_record(vaccine_reference_id);

CREATE INDEX IF NOT EXISTS idx_vaccine_recommendation_pet_id ON vaccine_recommendation(pet_id);
-- CREATE INDEX IF NOT EXISTS idx_vaccine_recommendation_vaccine_ref_id ON vaccine_recommendation(vaccine_reference_id); -- Comentado pois vaccine_reference_id não está mais na tabela

CREATE INDEX IF NOT EXISTS idx_booking_pet_id ON booking(pet_id);
CREATE INDEX IF NOT EXISTS idx_booking_booking_date ON booking(booking_date); -- Importante para filtros de data

CREATE INDEX IF NOT EXISTS idx_booking_recommendation_pet_id ON booking_recommendation(pet_id);

-- Índice na coluna de exclusão lógica para consultas rápidas de registros ativos
CREATE INDEX IF NOT EXISTS idx_tutor_enabled ON tutor(nenabled);
CREATE INDEX IF NOT EXISTS idx_pet_enabled ON pet(nenabled);
-- ... Adicione este índice em todas as tabelas onde a exclusão lógica será usada frequentemente em cláusulas WHERE.
"""

DML_SCRIPT = """
insert into organization_invite (invite_code, expiration_date) values
('super-secret-invite-code', NOW() + INTERVAL '30 days')

insert into organization (name, social_name, description, identification_code, links) values
('PetCare', 'PetCare Serviços Veterinários Ltda', 'Clínica veterinária especializada em cuidados para pets.', '12345678000199', ARRAY['http://localhost:3000'])
ON CONFLICT DO NOTHING;

insert into organization_apikey (organization_id, api_key) values
((SELECT organization_id FROM organization WHERE name='PetCare'), 'apikey-1234567890')
ON CONFLICT DO NOTHING;

INSERT INTO tutor (name, email, phone, organization_id) VALUES
('Ana Carolina', 'ana.carolina@email.com', '55 91234-5678', (SELECT organization_id FROM organization WHERE name='PetCare')),
('Bruno Martins', 'bruno.martins@email.com', '55 99876-5432', (SELECT organization_id FROM organization WHERE name='PetCare'))
ON CONFLICT (email) DO NOTHING;

INSERT INTO pet (tutor_id, name, species, animal_type, fur_type, birth_date) VALUES
(1, 'Bidu', 'Cão', 'Golden Retriever', 'Longo', NOW()),
(1, 'Luna', 'Gato', 'Siamês', 'Curto', NOW() - INTERVAL '1 year'),
(2, 'Thor', 'Cão', 'Shih Tzu', 'Longo', NOW() - INTERVAL '4 months')
ON CONFLICT DO NOTHING;

INSERT INTO booking (pet_id, service_type, booking_date, status) VALUES
(1, 'Banho e Tosa Completa', '2025-04-10 14:00:00', 'Realizado'),
(1, 'Banho e Tosa Completa', '2025-05-11 14:00:00', 'Realizado'),
(1, 'Banho e Tosa Completa', '2025-06-12 14:00:00', 'Realizado'),
(3, 'Banho', '2025-07-01 11:00:00', 'Realizado')
ON CONFLICT DO NOTHING;

INSERT INTO product (product_name, category) VALUES
('Ração para Cães de Pelo Longo', 'Alimentação'),
('Shampoo Hipoalergênico para Cães', 'Higiene')
ON CONFLICT DO NOTHING;

INSERT INTO purchase (tutor_id, product_id, purchase_date, quantity, price) VALUES
(1, 1, '2025-06-15', 1, 75.50),
(1, 2, '2025-06-15', 1, 30.00),
(2, 1, '2025-06-22', 1, 50.20)
ON CONFLICT DO NOTHING;

INSERT INTO vaccine_reference (vaccine_name, target_species, mandatory, description, equivalent_vaccines, first_dose_age_months, booster_interval_months) VALUES 
-- Aplicação Única 
('Aplicação de Microchip', 'Ambos', FALSE, 'Registro de aplicação de microchip de identificação.', NULL, 2, NULL),
 
-- Vacinas Não Essenciais para Ambos 
( 'Complexo Tosse dos Canis (Bordetella, Mucosa)', 'Ambos', FALSE, 'Vacinas vivas (intranasal ou oral) para proteção contra Bordetella bronchiseptica e/ou Parainfluenza.', NULL, 2, 12),
( 'Complexo Tosse dos Canis (Bordetella, Parenteral)', 'Ambos', FALSE, 'Vacina inativada (injetável) contra Bordetella bronchiseptica. Requer duas doses iniciais.', NULL, 2, 12),
( 'Borreliose de Lyme (Borrelia burgdorferi) canina', 'Ambos', FALSE, 'Recomendada para cães com alto risco de exposição a carrapatos em regiões onde a doença de Lyme é endêmica.', NULL, 2, 18),
( 'Gripe canina - Influenza Canina (H3N8 e H3N2)', 'Ambos', FALSE, 'Protege contra Bordetella bronchiseptica e/ou Parainfluenza. Considerar para cães em situações de risco, como canis, creches ou exposições.', NULL, 1.8, 12),
( 'Leishmaniose Canina', 'Ambos', FALSE, 'A vacinação é uma medida suplementar e não substitui o controle de flebotomíneos (vetores).', NULL, 1.8, 18), 

-- Vacinas Essenciais para Cães 
('V10 Canina (1 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 4 sorovares de Leptospirose.', 'V8 Canina', 1.45/* -- 45 dias / 31*/, NULL),  -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V10 Canina (2 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 4 sorovares de Leptospirose.', 'V8 Canina', 2.29/* -- (45 dias + 26 dias) = 71 dias / 31 dias*/, NULL), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado.
('V10 Canina (Dose Regular) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 4 sorovares de Leptospirose.', 'V8 Canina', 3.1/* -- (45 dias + 26 dias + 26 dias) = 97 dias / 31 dias*/, 12), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V8 Canina (1 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 2 sorovares de Leptospirose.', 'V10 Canina', 1.35/*-- 42 dias / 31 dias*/, NULL), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V8 Canina (2 Dose) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 2 sorovares de Leptospirose.', 'V10 Canina', 1.93/* -- (42 dias + 18 dias) / 31 dias*/, NULL), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
('V8 Canina (Dose Regular) - Polivalente Canina Essencial', 'Cão', TRUE, 'Protege contra Cinomose, Parvovirose, Hepatite, Adenovírus, Parainfluenza, Coronavirose e 2 sorovares de Leptospirose.', 'V10 Canina', 2.51/* -- (42 dias + 18 dias + 18 dias) / 31 dias*/, 12), -- No WSAVA falase em 36 meses para cães de baixo risco, mas aqui mantemos 12 meses conforme solicitado. 
( 'Raiva Canina - Antirrábica', 'Ambos', TRUE, 'Protege contra o vírus da Raiva. Obrigatória por lei no Brasil.', NULL, 3, 12),

-- Vacinas Não Essenciais para Cães 
('Giárdia Canina', 'Cão', FALSE, 'Protege contra Giardia lamblia.', NULL, 2, 18), -- Vacinas Essenciais para Gatos 
('V4 Felina (1 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose e Clamidiose.', 'V5 Felina', 1.77, NULL),
('V4 Felina (2 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose e Clamidiose.', 'V5 Felina', 2.74/* -- (55 dias + 30 dias) / 31 dias*/, NULL),
('V4 Felina (Dose Regular) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose e Clamidiose.', 'V5 Felina', 3.7/* -- (55 dias + 30 dias + 30 dias) / 31 dias*/, 12),
('V5 Felina (1 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose, Clamidiose e Leucemia Felina (FeLV).', 'V4 Felina', 2.25, NULL),
('V5 Felina (2 Dose) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose, Clamidiose e Leucemia Felina (FeLV).', 'V4 Felina', 3.61/* -- (70 dias + 21) / 31 dias*/, NULL),
('V5 Felina (Dose Regular) - Polivalente Essencial', 'Gato', TRUE, 'Protege contra Panleucopenia, Rinotraqueíte, Calicivirose, Clamidiose e Leucemia Felina (FeLV).', 'V4 Felina', 1.8/* -- (70 dias + 21 + 21) / 31 dias*/, 12), 

-- Vacinas Não Essenciais para Gatos 
('Bordetella Bronchiseptica Felina (Intranasal)', 'Gato', FALSE, 'Não utilizada rotineiramente. Considerar para gatos em colônias muito grandes.', NULL, 1, 12)
ON CONFLICT DO NOTHING;
"""

try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    print("Executando script DDL (criação de tabelas)...")
    cur.execute(DDL_SCRIPT)
    print("DDL executado com sucesso.")
    
    print("Executando script DML (inserção de dados)...")
    # Limpa as tabelas antes de inserir para garantir que o script seja idempotente
    cur.execute("""TRUNCATE TABLE
        execution_history,

        vaccination_record,
        vaccine_reference,
        vaccine_recommendation,

        booking,
        booking_reference,
        booking_recommendation,

        ltv_by_pet_profile,

        purchase,
        product,
        pet,
        tutor
        RESTART IDENTITY CASCADE;
    """)
    cur.execute(DML_SCRIPT)
    print("DML executado com sucesso.")
    
    conn.commit()
    print("Transação commitada.")
    
except psycopg2.OperationalError as e:
    print(f"Erro de conexão: {e}")
    print("Verifique se os contêineres Docker estão em execução ('docker-compose up -d') e se o nome do host do banco de dados ('{DB_HOST}') está correto.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()
        print("Conexão fechada.")

## 2. Verificando dados inseridos

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` para verificar se os dados foram inseridos corretamente na tabela `pet`.

In [ ]:
try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("SELECT * FROM pet;")
    rows = cur.fetchall()
    
    print("Registros encontrados:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()